In [ ]:
#hide
from utils import *
from IPython.display import display,HTML

# NLP deep dive: RNNs

## Language model fine tuning

## Preprocessing text with fastai

### Putting our texts into batches for a language model

### Tokenization

### Tokenization with fastai

In [ ]:
from fastai2.text.all import *
path = untar_data(URLs.IMDB)

In [ ]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [ ]:
txt = files[0].open().read(); txt[:75]

'This movie, which I just discovered at the video store, has apparently sit '

In [ ]:
spacy = SpacyTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#201) ['This','movie',',','which','I','just','discovered','at','the','video','store',',','has','apparently','sit','around','for','a','couple','of','years','without','a','distributor','.','It',"'s",'easy','to','see'...]


In [ ]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [ ]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#228) ['xxbos','xxmaj','this','movie',',','which','i','just','discovered','at','the','video','store',',','has','apparently','sit','around','for','a','couple','of','years','without','a','distributor','.','xxmaj','it',"'s",'easy'...]


In [ ]:
defaults.text_proc_rules

[<function fastai2.text.core.fix_html(x)>,
 <function fastai2.text.core.replace_rep(t)>,
 <function fastai2.text.core.replace_wrep(t)>,
 <function fastai2.text.core.spec_add_spaces(t)>,
 <function fastai2.text.core.rm_useless_spaces(t)>,
 <function fastai2.text.core.replace_all_caps(t)>,
 <function fastai2.text.core.replace_maj(t)>,
 <function fastai2.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [ ]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index'...]"

### Subword tokenization

In [ ]:
txts = [o.open().read() for o in files[:2000]]

In [ ]:
sp = SentencePieceTokenizer(vocab_sz=1000)

In [ ]:
sp.setup(txts)

In [ ]:
' '.join(first(sp([txt]))[:40])

'▁This ▁movie , ▁which ▁I ▁just ▁dis c over ed ▁at ▁the ▁video ▁st or e , ▁has ▁a p par ent ly ▁s it ▁around ▁for ▁a ▁couple ▁of ▁years ▁without ▁a ▁dis t ri but or . ▁It'

### Numericalization with fastai

In [ ]:
tfm = Tokenizer.from_folder(path)
files = get_text_files(path, folders = ['train', 'test', 'unsup'])
tls = TfmdLists(files, [tfm, Numericalize])
tls[0][:10]

tensor([  2,  19,  87, 236, 100, 910,  18, 170,  20,  91])

In [ ]:
tls.vocab[8:15]

['xxmaj', 'the', '.', ',', 'and', 'a', 'of']

In [ ]:
#hide
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tfm(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

In [ ]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

In [ ]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

In [ ]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

In [ ]:
dl = LMDataLoader(tls)

In [ ]:
tfms = [Tokenizer.from_folder(path), Numericalize]
files = get_text_files(path, folders = ['train', 'test', 'unsup'])
splits = RandomSplitter(valid_pct=0.1, seed=42)(files)
tls = TfmdLists(files, tfms, splits=splits)
dls = tls.dataloaders(dl_type=LMDataLoader)

### Putting our texts into batch for a classifier

In [ ]:
tfms = [[Tokenizer.from_folder(path), Numericalize], [parent_label, Categorize]]

In [ ]:
files = get_text_files(path, folders = ['train', 'test'])

In [ ]:
splits = GrandparentSplitter(valid_name='test')(files)

In [ ]:
dsets = Datasets(files, tfms, splits=splits)

In [ ]:
dls = dsets.dataloaders(dl_type=SortedDL, before_batch=pad_input_chunk)

### A layered API

In [ ]:
dls = TextDataLoaders.from_folder(path, valid='test')

In [ ]:
imdb = DataBlock(blocks=(TextBlock.from_folder(path),CategoryBlock),
                 get_y = parent_label,
                 get_items=partial(get_text_files, folders=['train', 'test']),
                 splitter=GrandparentSplitter(valid_name='test'))
dls = imdb.dataloaders(path)

In [ ]:
tfms = [[Tokenizer.from_folder(path), Numericalize], [parent_label, Categorize]]
files = get_text_files(path, folders = ['train', 'test'])
splits = GrandparentSplitter(valid_name='test')(files)
dsets = Datasets(files, tfms, splits=splits)
dls = dsets.dataloaders(dl_type=SortedDL, before_batch=pad_input_chunk)

## Training a text classifier

### Fine tuning the language model

In [ ]:
imdb_lm = DataBlock(blocks=(TextBlock.from_folder(path, is_lm=True),),
                    get_items=partial(get_text_files, folders=['train', 'test', 'unsup']),
                    splitter=RandomSplitter(0.1))

dls_lm = imdb_lm.dataloaders(path, path=path, bs=128, seq_len=80)

In [ ]:
dls_lm.show_batch(max_n=3)

In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 2e-2)

### Saving and loading models

In [ ]:
learn.save('1epoch')

In [ ]:
learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

In [ ]:
learn.save_encoder('finetuned')

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

i liked this movie because of its story and characters . The story line was very strong , very good for a sci - fi film . The main character , Alucard , was very well developed and brought the whole story
i liked this movie because i like the idea of the premise of the movie , the ( very ) convenient virus ( which , when you have to kill a few people , the " evil " machine has to be used to protect


### Fine tuning the classifier

In [ ]:
imdb_clas = DataBlock(blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
                      get_y = parent_label,
                      get_items=partial(get_text_files, folders=['train', 'test']),
                      splitter=GrandparentSplitter(valid_name='test'))
dls_clas = imdb_clas.dataloaders(path, path=path, bs=128, seq_len=80)

In [ ]:
dls_clas.show_batch(max_n=3)

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy).to_fp16()
learn = learn.load_encoder('finetuned')

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

## Disinformation and language models

## Questionnaire

### Further research

## Becoming a deep learning practitioner